In [319]:
import pandas as pd
import numpy as np

# Word2Vec 

In [320]:
# Process data
with open('/Users/dosmirnov/llm_task2/data/news.txt') as f:
    data = [x.split('\t')[-1] for x in f.read().split('\n')]

docs[0]

'Парусная гонка Giraglia Rolex Cup пройдет в Средиземном море в 64-й раз. Победители соревнования, проводимого с 1953 года Yacht Club Italiano, помимо других призов традиционно получают в подарок часы от швейцарского бренда Rolex. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру» в среду, 8 мая. Rolex Yacht-Master 40 Фото: пресс-служба Mercury Соревнования будут проходить с 10 по 18 июня. Первый этап: ночной переход из Сан-Ремо в Сен-Тропе 10-11 июня (дистанция 50 морских миль — около 90 километров). Второй этап: серия прибрежных гонок в бухте Сен-Тропе с 11 по 14 июня. Финальный этап пройдет с 15 по 18 июня: оффшорная гонка по маршруту Сен-Тропе — Генуя (243 морских мили — 450 километров). Маршрут проходит через скалистый остров Джиралья к северу от Корсики и завершается в Генуе.Регата, с 1997 года проходящая при поддержке Rolex, считается одной из самых значительных яхтенных гонок в Средиземноморье. В этом году в ней ожидается участие трех российских экипажей.'

In [321]:
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import nltk
from nltk.stem.snowball import SnowballStemmer




class Normalizer:
    def __init__(self):        
        self.stemmer = SnowballStemmer('russian')
        self.regexp_tokenizer = RegexpTokenizer(r'\w+')


    def normalize(self, text) -> list:
        words = self.regexp_tokenizer.tokenize(text)
        normalized = [self.stemmer.stem(word) for word in words]
        return normalized

normalizer = Normalizer()

normalizer.normalize('Привет, как тебя зовут?')

['привет', 'как', 'теб', 'зовут']

In [322]:
from functools import partial


class Word2VecTokenizer:
    def __init__(self, model=None):
        if model:
            self.model = model
            self.trained = True
        else:
            self.trained = False

        self.normalizer = Normalizer()

    def fit(self, corpus):
        print('Training word2vec model')
        sentences = [normalizer.normalize(sentence) for sentence in tqdm(corpus)]
        self.model = Word2Vec(sentences, vector_size=100, window=6, min_count=1, workers=4, sg=0)
        self.trained = True

    def _vectorize_with_pass(self, sentence):
        vectors = []
        for w in sentence:
            if w in self.model.wv:
                vectors.append(self.model.wv[w])
        return vectors
        

    def transform(self, text):
        if not self.trained:
            raise ValueError('Model should be trained first')

        normalized = self.normalizer.normalize(text)
        vectors = self._vectorize_with_pass(normalized)

        return vectors

    @staticmethod
    def aggregate(vectors, method='avg'):
        avg_functions = {
            'avg':partial(np.mean, axis=0),
            'max':partial(np.max, axis=0),
            'min':partial(np.min, axis=0)
        }
        return avg_functions[method](vectors)
        

w2v = Word2VecTokenizer()
w2v.fit(docs)

Training word2vec model


100%|████████████████████████████████████| 10001/10001 [00:27<00:00, 368.76it/s]


# Classifier

In [323]:
data = pd.read_csv('/Users/dosmirnov/llm_task2/data/news.txt', delimiter='\t', header=None, names=['label', 'header', 'text'])

In [324]:
data['vectors'] = data['text'].apply(w2v.transform)

In [326]:
data.head()

,label,header,text,vectors
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...,"[[-0.10808962, 0.25112814, 0.11802034, 0.19438..."
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...,"[[-0.008370347, 0.3923986, 0.3377372, 0.108342..."
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г...","[[0.10989315, -0.3886318, -0.1304021, 0.092096..."
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...,"[[1.1869445, -0.44310635, -2.2136948, -2.17737..."
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс...","[[1.004313, 0.21267386, -0.05910961, -0.183026..."


# Try avg aggregation

In [327]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train, X_test, y_train, y_test = train_test_split(
    list(data['vectors'].apply(partial(w2v.aggregate, method='avg'))),
    le.fit_transform(data['label']), 
    test_size=0.2, 
    random_state=42
)

In [328]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifier = SVC(gamma='auto')
classifier.fit(X_train, y_train)
f1_score(y_test, classifier.predict(X_test), average='micro')

0.759

# Try max/min aggregation

In [329]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train, X_test, y_train, y_test = train_test_split(
    list(data['vectors'].apply(partial(w2v.aggregate, method='max'))),
    le.fit_transform(data['label']), 
    test_size=0.2, 
    random_state=42
)

from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifier = SVC(gamma='auto')
classifier.fit(X_train, y_train)
f1_score(y_test, classifier.predict(X_test), average='micro')

0.5995

In [388]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train, X_test, y_train, y_test = train_test_split(
    list(data['vectors'].apply(partial(w2v.aggregate, method='min'))),
    le.fit_transform(data['label']), 
    test_size=0.2, 
    random_state=42
)

from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifier = SVC(gamma='auto')
classifier.fit(X_train, y_train)
f1_score(y_test, classifier.predict(X_test), average='micro')

0.6185

# Bonus: Recurrent networks

- Решение задачи векторного представления текста и классификации одной сущностью
- Контролируемость обучения 

In [392]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset


class BasicRNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(BasicRNNModel, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu', bidirectional = False)
        
        # Readout layer
        self.fc1 = nn.Linear(hidden_dim, output_dim)

    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out

# batch_size, epoch and iteration
batch_size = 100
n_iters = 8000
num_epochs = n_iters / (len(X_train) / batch_size)
num_epochs = int(num_epochs)

# Pytorch train and test sets
train = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
test = TensorDataset(torch.tensor(X_test),torch.tensor(y_test))

# data loader
train_loader = DataLoader(train, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
    
# Create RNN
input_dim = 100    # input dimension
hidden_dim = 256  # hidden layer dimension
layer_dim = 1   # number of hidden layers
output_dim = data['label'].nunique()   # output dimension

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# Adam Optimizer
# learning_rate = 0.0005
# weight_decay = 1e-4
optimizer = torch.optim.AdamW(model.parameters()) #, lr=learning_rate, weight_decay=weight_decay) 



In [ ]:
seq_dim = 1  
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
for epoch in range(num_epochs):
    for i, (texts, labels) in enumerate(train_loader):

        train  = Variable(texts.view(-1, seq_dim, input_dim))
        labels = Variable(labels )
            
        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = model(train)
        
        # Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        
        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        if count % 250 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for texts, labels in test_loader:
                texts = Variable(texts.view(-1, seq_dim, input_dim))
                
                # Forward propagation
                outputs = model(texts)
                
                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            if count % 500 == 0:
                # Print Loss
                print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.item(), accuracy))

In [391]:
f1_score(y_test, 
         torch.max(model(Variable(torch.tensor(X_test).view(-1, seq_dim, input_dim))), 1)[1],
        average='micro')

0.6045